In [1]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10948240511523478498
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 8602255617191097737
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2024-09-24 14:11:55.998021: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-09-24 14:11:55.998038: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-09-24 14:11:55.998042: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-09-24 14:11:55.998073: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-24 14:11:55.998085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
import tensorflow as tf

# Check for MPS support (Metal GPU backend)
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU not found. Trying MPS...")
    try:
        mps_devices = tf.config.experimental.list_physical_devices('MPS')
        if mps_devices:
            print("MPS device found:", mps_devices)
        else:
            print("MPS device not found")
    except Exception as e:
        print("Error:", e)


GPU is available


In [4]:
import os
import math
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [5]:
tf.__version__

'2.16.2'

In [6]:
# return gray image
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [7]:
# training targets
targets_df = pd.read_csv('data_csv/train.csv', header=None, sep=",", index_col=0)
targets = targets_df[1].to_dict()
print(targets[1157])

# validation targets
targets_validation_df = pd.read_csv('data_csv/validation.csv', header=None, sep=",", index_col=0)
targets_validation = targets_validation_df[1].to_dict()
print(targets_validation[8043])


Thumb Up
Thumb Up


In [8]:
targets_validation[8043]


'Thumb Up'

In [8]:
# classes label you want to use all labels 
'''label = pd.read_csv('data_csv/labels.csv',header=None, usecols=[0])
label.head()
targets_name = label[0].tolist()
len(targets_name)'''

# The classes (labels) we want to use
targets_name = [
    "Swiping Right",
    "Sliding Two Fingers Left",
    "No gesture",
    "Thumb Up"
    ]

In [9]:
len(targets_validation)

971

In [29]:
# Get the data directories
path = "training_samples/"
path_cv = "validation_samples/"

dirs = os.listdir(path)
dirs_cv = os.listdir(path_cv)

In [30]:
# number of samples for training and validation
print(len(dirs))
print(len(dirs_cv))

6000
971


In [13]:
training_count = 10


In [37]:
'''
The videos do not have the same number of frames, here we try to unify.
'''
hm_frames = 15 # number of frames
# unify number of frames for each training
def get_unify_frames(path):
    offset = 0
    # pick frames
    frames = os.listdir(path)
    frames_count = len(frames)
    # unify number of frames 
    if hm_frames > frames_count:
        # duplicate last frame if video is shorter than necessary
        frames += [frames[-1]] * (hm_frames - frames_count)
    elif hm_frames < frames_count:
        # If there are more frames, then sample starting offset
        #diff = (frames_count - hm_frames)
        #offset = diff-1 
        frames = frames[0:hm_frames]
    return frames  

In [12]:
# Resize frames
def resize_frame(frame):
    frame = img.imread(frame)
    frame = cv2.resize(frame, (64, 64))
    return frame
    

In [41]:
# Adjust training data
counter_training = 0 # number of training
training_targets = [] # training targets 
new_frames = [] # training data after resize & unify
for directory in dirs:
    new_frame = [] # one training
    # Frames in each folder
    frames = get_unify_frames(path+directory)
    if len(frames) == hm_frames: # just to be sure
        for frame in frames:
            frame = resize_frame(path+directory+'/'+frame)
            new_frame.append(rgb2gray(frame))
            if len(new_frame) == 15: # partition each training on two trainings.
                new_frames.append(new_frame) # append each partition to training data
                training_targets.append(targets_name.index(targets[int(directory)]))
                counter_training +=1
                new_frame = []

In [42]:
# we do the same for the validation data
counter_validation = 0
cv_targets = []
new_frames_cv = []
for directory in dirs_cv:
    new_frame = []
    # Frames in each folder
    frames = get_unify_frames(path_cv+directory)
    if len(frames)==hm_frames:
        for frame in frames:
            frame = resize_frame(path_cv+directory+'/'+frame)
            new_frame.append(rgb2gray(frame))
            if len(new_frame) == 15:
                new_frames_cv.append(new_frame)
                cv_targets.append(targets_name.index(targets_validation[int(directory)]))
                counter_validation +=1
                new_frame = []

In [20]:
# To check training length
print(len(new_frames))
print(len(training_targets))

NameError: name 'new_frames' is not defined

In [45]:
# To check validation length
print(len(new_frames_cv))
print(len(cv_targets))

1942
1942


In [46]:
training_targets[0:20]

[3, 3, 2, 2, 0, 0, 3, 3, 0, 0, 1, 1, 3, 3, 3, 3, 0, 0, 1, 1]

In [26]:
import numpy as np
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

def preprocess_frame(frame):
    """Resizes, converts to grayscale, and normalizes the frame."""
    frame = resize_frame(frame)  # Resize to 64x64
    frame = rgb2gray(frame)  # Convert to grayscale
    return frame

def frame_generator(dirs, path, targets, targets_name, batch_size=32):
    """Generator to yield batches of frames and corresponding targets."""
    scaler = StandardScaler()  # Initialize scaler

    while True:
        batch_frames = []  # Store batch of frames
        batch_targets = []  # Store batch of targets

        for directory in dirs:
            frames = get_unify_frames(os.path.join(path, directory))  # Unify frame count to 15 frames
            
            processed_frames = []
            for frame in frames:
                processed_frame = preprocess_frame(os.path.join(path, directory, frame))  # Preprocess each frame
                processed_frames.append(processed_frame)
            
            # Stack frames and reshape to (15, 64, 64, 1) for Conv3D
            processed_frames = np.stack(processed_frames).reshape(15, 64, 64, 1)

            # Normalize using StandardScaler across the frames
            processed_frames = scaler.fit_transform(processed_frames.reshape(-1, 64 * 64))  # Normalize pixel values
            processed_frames = processed_frames.reshape(15, 64, 64, 1)

            batch_frames.append(processed_frames)
            batch_targets.append(targets_name.index(targets[int(directory)]))

            # Once we have enough frames for a batch, yield them
            if len(batch_frames) == batch_size:
                yield np.array(batch_frames), np.array(batch_targets)
                batch_frames, batch_targets = [], []  # Reset for next batch

def create_dataset(dirs, path, targets, targets_name, batch_size=32):
    """Creates a tf.data.Dataset from the generator."""
    dataset = tf.data.Dataset.from_generator(
        lambda: frame_generator(dirs, path, targets, targets_name, batch_size),
        output_signature=(
            tf.TensorSpec(shape=(batch_size, 15, 64, 64, 1), dtype=tf.float32),
            tf.TensorSpec(shape=(batch_size,), dtype=tf.int32)
        )
    )

    # Shuffle, batch, and prefetch
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset



In [1]:
#show data
fig = plt.figure()
for i in range(2,4):
    for num,frame in enumerate(new_frames[i][0:18]):
        y = fig.add_subplot(4,5,num+1)
        y.imshow(frame, cmap='gray')
    fig = plt.figure()
plt.show()

NameError: name 'plt' is not defined

In [35]:
# convert training data to np float32
training_data = np.array(new_frames[0:counter_training], dtype=np.float32)

NameError: name 'new_frames' is not defined

In [49]:
# Function to empty the RAM
def release_list(a):
   del a[:]
   del a

In [24]:
release_list(new_frames)

In [50]:
# convert validation data to np float32
cv_data = np.array(new_frames_cv[0:counter_validation], dtype=np.float32)

In [51]:
release_list(new_frames_cv)

In [52]:
training_data.shape

(12000, 15, 64, 64)

In [53]:
cv_data.shape

(1942, 15, 64, 64)

In [54]:
# Normalisation: training
print('old mean', training_data.mean())
scaler = StandardScaler()
scaled_images  = scaler.fit_transform(training_data.reshape(-1, 15*64*64))
print('new mean', scaled_images.mean())
scaled_images  = scaled_images.reshape(-1, 15, 64, 64, 1)
print(scaled_images.shape)

old mean 115.14039
new mean -4.5216747e-10
(12000, 15, 64, 64, 1)


In [55]:
# Normalisation: validation
print('old mean', cv_data.mean())
scaler = StandardScaler()
scaled_images_cv  = scaler.fit_transform(cv_data.reshape(-1, 15*64*64))
print('new mean',scaled_images_cv.mean())
scaled_images_cv  = scaled_images_cv.reshape(-1, 15, 64, 64, 1)
print(scaled_images_cv.shape)

old mean 111.76559
new mean -1.5796356e-08
(1942, 15, 64, 64, 1)


In [27]:
class Conv3DModel(tf.keras.Model):
    def __init__(self):
        super(Conv3DModel, self).__init__()
        # Convolutions
        self.conv1 = tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
        self.conv2 = tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv2", data_format='channels_last')
        self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
        
        # LSTM & Flatten
        self.convLSTM = tf.keras.layers.ConvLSTM2D(40, (3, 3))
        self.flatten = tf.keras.layers.Flatten(name="flatten")

        # Dense layers
        self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
        self.out = tf.keras.layers.Dense(4, activation='softmax', name="output")

    def call(self, x):
        # Reshape input: ensure (batch_size, 15, 64, 64, 1)
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.convLSTM(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.out(x)

# Input tensor shape should be (batch_size, 15, 64, 64, 1)


In [17]:
# My model
class Conv3DModel(tf.keras.Model):
  def __init__(self):
    super(Conv3DModel, self).__init__()
    # Convolutions
    self.conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
    self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
    self.conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
    self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')
   
    # LSTM & Flatten
    self.convLSTM = tf.keras.layers.ConvLSTM2D(40, (3, 3), padding='same', return_sequences=False)
    # self.convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
    self.flatten =  tf.keras.layers.Flatten(name="flatten")

    # Dense layers
    self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
    #self.out = tf.keras.layers.Dense(4, activation='softmax', name="output")
    self.out = tf.keras.layers.Dense(4, activation='softmax')

    

  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.convLSTM(x)
    #x = self.pool2(x)
    #x = self.conv3(x)
    #x = self.pool3(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.out(x)

In [24]:
batch_size = 32
# Create the training dataset
train_dataset = create_dataset(dirs, path, targets, targets_name, batch_size)

# Create the validation dataset
val_dataset = create_dataset(dirs_cv, path, targets_validation, targets_name, batch_size)


In [39]:
# Create the dataset
batch_size = 1000
# Create the training dataset
train_dataset = create_dataset(dirs, path, targets, targets_name, batch_size)

# Create the validation dataset
val_dataset = create_dataset(dirs_cv, path_cv, targets_validation, targets_name, batch_size)

# Compile the model
model = Conv3DModel()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using the dataset
with tf.device('/device:GPU:0'):
    history = model.fit(
        train_dataset, 
        epochs=10, 
        steps_per_epoch=len(dirs) // batch_size,
        validation_data=val_dataset,
        validation_steps=len(dirs_cv) // batch_size
    )


Epoch 1/10


2024-09-24 14:56:59.683226: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:150: Filling up shuffle buffer (this may take a while): 2 of 1000
2024-09-24 14:57:12.144369: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:150: Filling up shuffle buffer (this may take a while): 4 of 1000
2024-09-24 14:57:29.697969: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:150: Filling up shuffle buffer (this may take a while): 7 of 1000
2024-09-24 14:57:40.983078: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:150: Filling up shuffle buffer (this may take a while): 9 of 1000
2024-09-24 14:57:58.160604: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:150: Filling up shuffle buffer (this may take a while): 12 of 1000


In [57]:
model = Conv3DModel()

In [58]:
# choose the loss and optimizer methods
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [59]:
x_train = np.array(scaled_images)
y_train = np.array(training_targets)
x_val = np.array(scaled_images_cv)
y_val = np.array(cv_targets)

In [60]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(12000, 15, 64, 64, 1)
(12000,)
(1942, 15, 64, 64, 1)
(1942,)


#### here I use the checkpoints
read more:
https://www.tensorflow.org/beta/guide/checkpoints

In [63]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_today/cp-{epoch:04d}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True)

In [65]:
# Run the training 
history = model.fit(x_train, y_train,
                    callbacks = [cp_callback],
                    validation_data=(x_val, y_val),
                    batch_size=32,
                    epochs=1)

272/375 ━━━━━━━━━━━━━━━━━━━━ 2:13 1s/step - accuracy: 0.5297 - loss: 1.0047

KeyboardInterrupt: 

In [66]:
# just after one epoch
history.history

NameError: name 'history' is not defined

In [41]:
# save the model for use in the application
model.save_weights('weights/path_to_my_weights', save_format='tf')

## 2. use a more complex syntax, for experts


In [ ]:
# My model
class Conv3DModel(tf.keras.Model):
  def __init__(self):
    super(Conv3DModel, self).__init__()
    # Convolutions
    self.conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
    self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
    self.conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
    self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')
   
    # LSTM & Flatten
    self.convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
    self.flatten =  tf.keras.layers.Flatten(name="flatten")

    # Dense layers
    self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
    self.out = tf.keras.layers.Dense(4, activation='softmax', name="output")
    

  def call(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = self.convLSTM(x)
    #x = self.pool2(x)
    #x = self.conv3(x)
    #x = self.pool3(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.out(x)

In [ ]:
model = Conv3DModel()

In [41]:
# use tensorflow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((scaled_images, training_targets))
cv_dataset = tf.data.Dataset.from_tensor_slices((scaled_images_cv, cv_targets))

In [45]:
model(scaled_images[0:2])

<tf.Tensor: id=443, shape=(2, 4), dtype=float32, numpy=
array([[0.22843026, 0.25249583, 0.25405762, 0.26501626],
       [0.23236132, 0.2505814 , 0.2503527 , 0.26670453]], dtype=float32)>

In [46]:
model.summary()

Model: "conv3d_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               multiple                  896       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 multiple                  0         
_________________________________________________________________
conv1 (Conv3D)               multiple                  55360     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 multiple                  0         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  multiple                  149920    
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
d1 (Dense)                   multiple               

In [49]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [204]:
# Loss
train_loss = tf.keras.metrics.Mean(name='train_loss')
valid_loss = tf.keras.metrics.Mean(name='valid_loss')
# Accuracy
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [205]:
@tf.function
def train_step(image, targets):
    with tf.GradientTape() as tape:
        # Make a prediction on all the batch
        predictions = model(image)
        # Get the error/loss on these predictions
        loss = loss_fn(targets, predictions)
    # Compute the gradient which respect to the loss
    grads = tape.gradient(loss, model.trainable_variables)
    # Change the weights of the model
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # The metrics are accumulate over time. You don't need to average it yourself.
    train_loss(loss)
    train_accuracy(targets, predictions)

In [206]:
@tf.function
def valid_step(image, targets):
    predictions = model(image)
    t_loss = loss_fn(targets, predictions)
    # Set the metrics for the test
    valid_loss(t_loss)
    valid_accuracy(targets, predictions)

#### here I use the checkpoints
read more:
https://www.tensorflow.org/beta/guide/checkpoints

In [207]:
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(ckpt, 'training_checkpoints/tf_ckpts', max_to_keep=10)
ckpt.restore(manager.latest_checkpoint)

In [ ]:
epoch = 10
batch_size = 32
b = 0
training_acc = []
validation_acc = []
for epoch in range(epoch):
    # Training set
    for images_batch, targets_batch in train_dataset.batch(batch_size):
        train_step(images_batch, targets_batch)
        template = '\r Batch {}/{}, Loss: {}, Accuracy: {}'
        print(template.format(
            b, len(training_targets), train_loss.result(), 
            train_accuracy.result()*100
        ), end="")
        b += batch_size
    # Validation set
    for images_batch, targets_batch in cv_dataset.batch(batch_size):
        valid_step(images_batch, targets_batch)

    template = '\nEpoch {}, Valid Loss: {}, Valid Accuracy: {}'
    print(template.format(
        epoch+1,
        valid_loss.result(), 
        valid_accuracy.result()*100)
    )
    training_acc.append(float(train_accuracy.result()*100))
    validation_acc.append(float(valid_accuracy.result()*100))
    ckpt.step.assign_add(1)
    save_path = manager.save()
    print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    train_accuracy.reset_states()
    train_loss.reset_states()

In [209]:
print(manager.checkpoints)

['training_checkpoints/tf_ckpts\\ckpt-1', 'training_checkpoints/tf_ckpts\\ckpt-2', 'training_checkpoints/tf_ckpts\\ckpt-3', 'training_checkpoints/tf_ckpts\\ckpt-4', 'training_checkpoints/tf_ckpts\\ckpt-5', 'training_checkpoints/tf_ckpts\\ckpt-6', 'training_checkpoints/tf_ckpts\\ckpt-7', 'training_checkpoints/tf_ckpts\\ckpt-8', 'training_checkpoints/tf_ckpts\\ckpt-9']


In [ ]:
# plote Accuracy / epoch
plt.plot([1,2,3,4,5,6,7,8,9],training_acc, '-' )
plt.plot([1,2,3,4,5,6,7,8,9],validation_acc, '-' )

plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.show()

In [217]:
# save the model for use in the application
model.save_weights('weights/path_to_my_weights', save_format='tf')